In [2]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [3]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
import matplotlib as mt
train_datagen=ImageDataGenerator()

test_datagen= ImageDataGenerator(rescale=1./255)



In [4]:
batch_size=10
train_data=train_datagen.flow_from_directory(
'C:/Users/anish/Documents/GitHub/LetsUpggrade-AI-ML/Project/Project 10/data/train/,
target_size=(150,150),
batch_size=batch_size,
class_mode='binary')

Found 1315 images belonging to 2 classes.


In [6]:
test_data=test_datagen.flow_from_directory(
'C:/Users/anish/Documents/GitHub/LetsUpggrade-AI-ML/Project/Project 10/data/train/',
target_size=(150,150),
batch_size=batch_size,
class_mode='binary')

Found 194 images belonging to 2 classes.


In [7]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [12]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',
                             verbose=0,save_best_only=True,mode='auto')

In [13]:
histroy=model.fit_generator(
                            train_data,steps_per_epoch=50//batch_size,epochs=10,
                             validation_data=test_data,validation_steps=40//batch_size)

C:\Users\Tinesh\anaconda3\envs\baseclone\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
5/5 [==============================] - 8s 2s/step - loss: 281.5871 - acc: 0.6033 - val_loss: 0.6730 - val_acc: 0.6000
Epoch 2/10
5/5 [==============================] - 7s 1s/step - loss: 14.6981 - acc: 0.4294 - val_loss: 0.6930 - val_acc: 0.5000
Epoch 3/10
5/5 [==============================] - 8s 2s/step - loss: 0.9771 - acc: 0.5406 - val_loss: 0.6937 - val_acc: 0.4500
Epoch 4/10
5/5 [==============================] - 8s 2s/step - loss: 0.7854 - acc: 0.4617 - val_loss: 0.6927 - val_acc: 0.5750
Epoch 5/10
5/5 [==============================] - 7s 1s/step - loss: 0.6138 - acc: 0.6511 - val_loss: 0.6930 - val_acc: 0.5000
Epoch 6/10
5/5 [==============================] - 7s 1s/step - loss: 0.7607 - acc: 0.6672 - val_loss: 0.6920 - val_acc: 0.6250
Epoch 7/10
5/5 [==============================] - 7s 1s/step - loss: 0.6559 - acc: 0.6147 - val_loss: 0.6922 - val_acc: 0.5750
Epoch 8/10
5/5 [==============================] - 7s 1s/step - loss: 0.7498 - acc: 0.6914 - val_loss: 0.6926

In [14]:
model.save('mask_project.h5')

In [15]:
import cv2
import numpy as np
from keras.models import load_model

In [16]:
import tensorflow as tf

In [17]:
model = tf.keras.models.load_model('mask_project.h5')

In [21]:
import cv2
import numpy as np
from keras.models import load_model
model = tf.keras.models.load_model('mask_project.h5')

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('C:/Users/anish/Documents/GitHub/LetsUpggrade-AI-ML/Project/Project 10/haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()